In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import sys
import os
import argparse
from glob import glob

import random
from random import shuffle

import time
import datetime

from collections import Counter

# from numpy.random import seed
# seed(101)
# from tensorflow import set_random_seed
# set_random_seed(101)


import numpy as np
import pandas as pd

import shutil
from tqdm import tqdm

import itertools


# import inspect
import gc

import re

from PIL import Image
import cv2

import keras

# from keras.utils import np_utils

from keras.preprocessing.image import ImageDataGenerator

# from imgaug import augmenters as iaa
# import imgaug as ia


from keras import models
from keras.models import Model
from keras.models import Sequential


from keras.layers import Convolution1D, concatenate, SpatialDropout1D, GlobalMaxPool1D, GlobalAvgPool1D, Embedding, \
    Conv2D, SeparableConv1D, Add, BatchNormalization, Activation, GlobalAveragePooling2D, LeakyReLU, Flatten
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, \
    Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D
from keras.layers.pooling import _GlobalPooling1D


from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.densenet import DenseNet201
from keras.applications.nasnet import NASNetMobile, NASNetLarge

from keras.applications.nasnet import preprocess_input



# from keras.constraints import maxnorm


from keras import optimizers
from keras.optimizers import Adam, SGD , RMSprop

from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy


from keras import backend as K
# K.set_image_dim_ordering('th')


from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

# from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight as cw


from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report


from mlxtend.plotting import plot_confusion_matrix

import tensorflow as tf



from IPython.display import display

import seaborn as sns

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline


In [ ]:
print(os.listdir(r"../input"))


In [ ]:
input_directory = r"../input/"

training_dir = input_directory + r"train"
testing_dir = input_directory + r"test"


In [ ]:
train_files = os.listdir(training_dir)
test_files = os.listdir(testing_dir)

train_labels = []

for file in train_files:
    train_labels.append(file.split(".")[0])
    
df_train = pd.DataFrame({"id": train_files, "label": train_labels})

df_train.head()


In [ ]:
test_ids = []
test_labels = []
for file in test_files:
    test_ids.append(int(file.split(".")[0]))
    test_labels.append(-1)
    
df_test_1 = pd.DataFrame({"id": test_files, "label": test_labels})

df_test = pd.DataFrame({"id": test_ids, "label": test_labels})
df_test['id'] = df_test.id.astype('int32')
df_test['label'] = df_test.label.astype('int32')
df_test.head()


In [ ]:
# reset tensorflow graph tp free up memory and resource allocation 
def reset_graph(model=None):
    if model:
        try:
            del model
        except:
            return False
    
    tf.reset_default_graph()
    
    K.clear_session()
    
    gc.collect()
    
    return True


# reset callbacks 
def reset_callbacks(checkpoint=None, reduce_lr=None, early_stopping=None, tensorboard=None):
    checkpoint = None
    reduce_lr = None
    early_stopping = None
    tensorboard = None
    


In [ ]:
# reset_graph()
# reset_callbacks()


In [ ]:
def get_data(batch_size=32, target_size=(96,96), class_mode="categorical", training_dir=training_dir, testing_dir=testing_dir, df_train=df_train, df_test=df_test):
    print("Generating data following preprocessing...")
    
    rescale = 1.0/255

    train_batch_size = batch_size
    test_batch_size = batch_size

    train_shuffle = True
    val_shuffle = True
    test_shuffle = False
    
    train_datagen = ImageDataGenerator(
        horizontal_flip=True,
#         vertical_flip=True,
#         rotation_range=45,
        shear_range=0.2,
        zoom_range=0.2,
        rescale=rescale,
        validation_split=0.3
    )

    train_generator = train_datagen.flow_from_dataframe(
        df_train, 
        training_dir, 
        x_col='id',
        y_col='label', 
        has_ext=True, 
        target_size=target_size, 
        class_mode=class_mode, 
        batch_size=batch_size, 
        shuffle=True, 
        seed=42,
        subset='training')
    
    validation_generator = train_datagen.flow_from_dataframe(
        df_train, 
        training_dir, 
        x_col='id',
        y_col='label', 
#         has_ext=True, 
        target_size=target_size, 
        class_mode=class_mode, 
        batch_size=batch_size, 
        shuffle=True, 
        seed=42,
        subset='validation')


    test_datagen = ImageDataGenerator(rescale=rescale)
    test_generator = test_datagen.flow_from_dataframe(
        df_test, 
        testing_dir, 
        x_col='id',
        y_col='label', 
        has_ext=True, 
        target_size=target_size, 
        class_mode=class_mode, 
        batch_size=batch_size, 
        shuffle=True, 
        seed=42)
    
    class_weights = get_weight(train_generator.classes)
    
    steps_per_epoch = len(train_generator)
    validation_steps = len(validation_generator)
    
    print("Data batches generated.")
    
    return train_generator, validation_generator, test_generator, class_weights, steps_per_epoch, validation_steps


def get_weight(y):
    class_weight_current =  cw.compute_class_weight('balanced', np.unique(y), y)
    return class_weight_current


In [ ]:
def get_model(model_name, input_shape=(96, 96, 3)):
    inputs = Input(input_shape)
    
    if model_name == "Xception":
        base_model = Xception(include_top=False, input_shape=input_shape)
    elif model_name == "ResNet50":
        base_model = ResNet50(include_top=False, input_shape=input_shape)
    elif model_name == "InceptionV3":
        # base_model = InceptionV3(include_top=False, input_shape=input_shape)
        # included weights
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape) 
    elif model_name == "InceptionResNetV2":
        base_model = InceptionResNetV2(include_top=False, input_shape=input_shape)
    if model_name == "DenseNet201":
        base_model = DenseNet201(include_top=False, input_shape=input_shape)
    if model_name == "NASNetMobile":
        base_model = NASNetMobile(include_top=False, input_shape=input_shape)
    if model_name == "NASNetLarge":
        base_model = NASNetLarge(include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    model.summary()
    
    return model


In [ ]:
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


In [ ]:
def plot_performance(history=None):
    plt.figure(figsize=(12,4))
    # Plot training & validation accuracy values
    plt.subplot(121)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()


In [ ]:
main_model_dir = r"models/"
main_log_dir = r"logs/"

try:
    shutil.rmtree(main_model_dir)
except:
    pass
try:
    shutil.rmtree(main_log_dir)
except:
    pass

os.mkdir(main_model_dir)
os.mkdir(main_log_dir)


In [ ]:
model_dir = main_model_dir + time.strftime('%Y-%m-%d %H-%M-%S') + "/"
log_dir = main_log_dir + time.strftime('%Y-%m-%d %H-%M-%S')

os.mkdir(model_dir)
os.mkdir(log_dir)

model_file = model_dir + "{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"


In [ ]:
# reset_graph()
# reset_callbacks()


In [ ]:
print("Settting Callbacks")

checkpoint = ModelCheckpoint(
    model_file, 
    monitor='val_acc', 
    save_best_only=True)


tensorboard = TensorBoard(
    log_dir=log_dir,
    update_freq='batch')


early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=1,
    verbose=1,
    restore_best_weights=True)


reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    verbose=1)

callbacks = [reduce_lr, early_stopping, checkpoint]

print("Completed")


In [ ]:
# print("Getting Base Model...")
# model = get_model("Xception")
# model = get_model("ResNet50")
# model = get_model("InceptionV3")
# model = get_model("InceptionResNetV2")
# model = get_model("DenseNet201")
# model = get_model("NASNetMobile")
# model = get_model("NASNetLarge")
# print("complete")


In [ ]:
print("Starting...\n")
start_time = time.time()
# print(date_time(1))

batch_size = 32
target_size = (299, 299)
df_test=df_test_1

train_generator, validation_generator, test_generator, class_weights, steps_per_epoch, validation_steps = get_data(batch_size=batch_size, target_size=target_size, df_test=df_test)
print("\n\nCompleted ...\n")


In [ ]:
loss = 'categorical_crossentropy'
metrics = ['acc']


In [ ]:
print("Starting...\n")
start_time = time.time()
# print(date_time(1))


# create the base pre-trained model
# input_shape = (96, 96, 3)
# base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

print("\n\nCompliling Model ...\n")
learning_rate = 0.0001
optimizer = Adam(learning_rate)
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# train the model on the new data for a few epochs
verbose = 1
epochs = 1

print("Trainning Model ...\n")
history = model.fit_generator(
    train_generator,
    steps_per_epoch=1,
    epochs=1,
    verbose=verbose,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=1, 
    class_weight=class_weights)


elapsed_time = time.time() - start_time
elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

print("\nElapsed Time: " + elapsed_time)
# print("Completed\n", date_time(1))
print("\n")


In [ ]:
# print("Starting...\n")
# start_time = time.time()
# # print(date_time(1))


# # for i, layer in enumerate(base_model.layers):
# #    print(i, layer.name)

# for layer in model.layers[:249]:
#    layer.trainable = False
# for layer in model.layers[249:]:
#    layer.trainable = True


# print("\n\nCompliling Model ...\n")
# learning_rate = 0.0001
# optimizer = Adam(learning_rate)
# model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# verbose = 1
# epochs = 2

# print("Trainning Model ...\n")
# history = model.fit_generator(
#     train_generator,
#     steps_per_epoch=steps_per_epoch,
#     epochs=epochs,
#     verbose=verbose,
#     callbacks=callbacks,
#     validation_data=validation_generator,
#     validation_steps=validation_steps, 
#     class_weight=class_weights)


# elapsed_time = time.time() - start_time
# elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

# print("\nElapsed Time: " + elapsed_time)
# # print("Completed\n", date_time(1))
# print("\n")


In [ ]:
# plot_performance(history=history)


In [ ]:
def generate_result(model, test_generator, nsteps=len(test_generator)):
    y_preds = model.predict_generator(test_generator, steps=nsteps, verbose=1) 
    return y_preds[:,1]


In [ ]:
y_preds = generate_result(model, test_generator)


In [ ]:
test_ids = []
test_labels = []
for file in test_files:
    test_ids.append(int(file.split(".")[0]))
    test_labels.append(-1)

df_test = pd.DataFrame({"id": test_ids, "label": test_labels})
df_test['id'] = df_test.id.astype('int32')
df_test['label'] = df_test.label.astype('int32')
df_test.head()


In [ ]:
submission_csv = "submission.csv"
df_test['label'] = y_preds
df_test = df_test.sort_values('id')
df_test.to_csv(submission_csv, index=False)

df_test.head()
